In [2]:
from audioFunctions import *
from binaryFunctions import *
from qam import *
from sequenceGenerators import *
from receiver import *
from to_import import *

In [3]:
def str_to_bytearray(string_data):
    new_data = []
    for i in range(0, len(string_data), 8):
        new_data.append(string_data[i:i+8])  

    int_data = [] 
    for i in new_data:
        int_data.append(int(i,2))

    return bytearray(int_data)

In [7]:
with open('Data_Files/a7r56tu_knownseq.csv', 'r') as file:
    raw = file.read().replace('\n', ',').split(',')

known = []
for value in raw:
    try:
        v = float(value)
        if v == 0:
            symbol = cmath.exp((math.pi/4) * 1j)
        elif v ==1:
            symbol = cmath.exp((3*math.pi/4) * 1j)
        elif v == 2:
            symbol = cmath.exp((5*math.pi/4) * 1j)
        else:
            symbol = cmath.exp((7*math.pi/4) * 1j)
        

        known.append(symbol)
    except:
        pass
    
known = np.array(known)

In [21]:
first1024 = known[:1024]

first_half = []
for val in first1024:
    first_half.append(val)
    first_half.append(0)
first_half.pop()

known_fourier = np.concatenate(([0], first_half, [0], np.conjugate(first_half[::-1])))
known_time = ifft(known_fourier).real

0.006905339660024858
-0.006905339660024858


In [23]:
y = audioDataFromFile('Data_Files/b8v89t_received.wav')

In [34]:
y_section = np.reshape(y, y.size)[0:200000]
g = np.correlate(y_section, known_time, 'valid') # convoluton
i_max = np.argmax(g[:int(len(g)/2)])

In [32]:
rng = 2
#plot_y(g[i_max - rng:i_max + rng], f=2, title="Closeup of peak +- " + str(rng) + " samples")

In [49]:
y_dash = y[i_max + len(known_time):]

In [50]:
d = len(y_dash)/4096

chunks = []
for i in range(int(d)):
    chunks.append(y_dash[i*4096:(i+1)*4096])

In [51]:
# FIRST ROUND
binary = []
for i in range(1):  #len(chunks)):
    fft_data = fft(chunks[i])[1:2048]
    
    #plot_y(fft_data.imag, f = i)
    #plot_y(chunks[i][1:2048], f = i)
    
    for freq_bin in fft_data:
        
        #print(freq_bin)

        bit0 = 0
        bit1 = 0

        if freq_bin.real < 0:
            bit1 = 1
        if freq_bin.imag < 0:
            bit0 = 1

        binary.append(bit0)
        binary.append(bit1)

bin_single_sting = ""
for i in range(len(binary)):
    bin_single_sting += str(binary[i])
    
print(bin_single_sting[8*28:8*29])
str_len = len(bin_single_sting)
print(str_len)
a = fft(chunks[0])[1:]

01010110
4094


In [52]:
# Find information about the file name and the byte length of the file
file_seperation = []
count = 0
for i in range(math.floor(len(binary)/8)):
    byte = binary[i*8:(i+1)*8]
    zero = True
    for bit in byte:
        if bit == 1:
            zero = False
            
    if zero:
        #print(byte, "zero")
        file_seperation.append(i*8)
            
        count += 1
    
    if count == 2:
        break
        
print(file_seperation)
# Extract file name and length of the file
Filename= str_to_bytearray(bin_single_sting[0:file_seperation[0]]).decode()
Length = int(str_to_bytearray(bin_single_sting[file_seperation[0]+8:file_seperation[1]]).decode())

extra_bits = len(binary) - file_seperation[1] - 8 - Length*8

raw_file = bin_single_sting[file_seperation[1] + 8 : -1*extra_bits]

print(Filename, Length, extra_bits)

[80, 136]
b8v89t.wav 761956 -6091698


In [53]:
loop = math.ceil(Length * (4/2047))
binary_long = []
for i in range(loop):  #len(chunks)):
    if i%100 == 0:
        print(i, " ", end = '')
    fft_data = fft(chunks[i])[1:2048]
    
    
    for j, freq_bin in enumerate(fft_data):

        bit0 = 0
        bit1 = 0

        if freq_bin.real < 0:
            bit1 = 1
        if freq_bin.imag < 0:
            bit0 = 1

        binary_long.append(bit0)
        binary_long.append(bit1)

bin_single_sting_long = ""
for i in range(len(binary_long)):
    bin_single_sting_long += str(binary_long[i])
    
print(bin_single_sting_long[8*28:8*29])
str_len_long = len(bin_single_sting_long)
print(str_len_long)

0  100  200  300  400  500  600  700  800  900  1000  1100  1200  1300  1400  01010110
6095966


In [54]:
# Find information about the file name and the byte length of the file
file_seperation = []
count = 0
for i in range(math.floor(len(binary_long)/8)):
    byte = binary_long[i*8:(i+1)*8]
    zero = True
    for bit in byte:
        if bit == 1:
            zero = False
            
    if zero:
        #print(byte, "zero")
        file_seperation.append(i*8)
            
        count += 1
    
    if count == 2:
        break
        
# Extract file name and length of the file
Filename= str_to_bytearray(bin_single_sting_long[0:file_seperation[0]]).decode()
Length = int(str_to_bytearray(bin_single_sting_long[file_seperation[0]+8:file_seperation[1]]).decode())

extra_bits = len(binary_long) - file_seperation[1] - 8 - Length*8

raw_file = bin_single_sting_long[file_seperation[1] + 8 : -1*extra_bits]

raw_file_length = len(raw_file)
print(Filename, Length, extra_bits, raw_file_length)

b8v89t.wav 761956 174 6095648


In [55]:
string_data = raw_file

new_data = []
for i in range(0, len(string_data), 8):
    new_data.append(string_data[i:i+8])  

int_data = [] 
for i in new_data:
    int_data.append(int(i,2))

byte = bytearray(int_data)

In [56]:
newFile = open("Data_Files/w2_2_2_key.wav", mode = 'bx')
newFile.write(byte)

761956

In [47]:
string_data = raw_file

new_data = []
for i in range(0, len(string_data), 8):
    new_data.append(string_data[i:i+8])  

int_data = [] 
for i in new_data:
    int_data.append(int(i,2))

byte = bytearray(int_data)

In [48]:
newFile = open("Data_Files/key.gif", "wb")
newFile.write(byte)

1476376

In [15]:
with open('Data_Files/a7r56tu_knownseq.csv', 'r') as file:
    raw = file.read().replace('\n', ',').split(',')

known = []
for value in raw:
    try:
        v = float(value)
        if v == 0:
            symbol = cmath.exp((math.pi/4) * 1j)
        elif v ==1:
            symbol = cmath.exp((3*math.pi/4) * 1j)
        elif v == 2:
            symbol = cmath.exp((5*math.pi/4) * 1j)
        else:
            symbol = cmath.exp((7*math.pi/4) * 1j)
        

        known.append(symbol)
    except:
        pass
    
known = np.array(known)

In [16]:
known_fourier = np.concatenate(([0], known, [0], np.conjugate(known[::-1])))
known_time = ifft(known_fourier).real

In [70]:
a = [2,3,4,5]
print(a[1:3])

[3, 4]
